# Ulepszone Animacje FMR Modów

Ten notebook demonstruje nową ulepszoną implementację animacji FMR modów z:

## ✨ Nowe funkcje:
- **Prawdziwa fizyczna animacja**: `amplitude * cos(phase + ωt)` - pokazuje rzeczywistą dynamikę modu
- **Mapy kolorów cmocean**: `balance`, `diff`, `curl`, `delta`, `tarn` - specjalistyczne mapy naukowe
- **MidpointNormalize**: Symetryczna normalizacja wokół zera dla danych oscylujących
- **Inteligentne domyślne**: Automatyczny wybór najlepszych ustawień dla każdego typu animacji

## 🎯 Typy animacji:
1. **`temporal`**: Oscylacje w czasie przy stałej częstotliwości (fizycznie prawdziwe)
2. **`frequency`**: Zmiana amplitudy w funkcji częstotliwości 
3. **`phase`**: Ewolucja fazy przy stałej częstotliwości

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from mmpp import MMPP
from mmpp.fft.modes import FMRModeAnalyzer, ModeVisualizationConfig, MidpointNormalize

# Sprawdź czy cmocean jest dostępne
try:
    import cmocean

    print("✅ cmocean dostępne - używając naukowych map kolorów")
    CMOCEAN_AVAILABLE = True

    # Pokaż dostępne mapy kolorów
    scientific_colormaps = ["balance", "diff", "curl", "delta", "tarn"]
    print(f"📊 Dostępne mapy cmocean: {', '.join(scientific_colormaps)}")
except ImportError:
    print("⚠️  cmocean niedostępne - używając matplotlib")
    CMOCEAN_AVAILABLE = False

## Załaduj dane

In [ ]:
# Załaduj swoje dane MMPP
zarr_path = "/path/to/your/data.zarr"  # ⚠️ ZMIEŃ NA SWOJĄ ŚCIEŻKĘ

try:
    op = MMPP(zarr_path, debug=True)
    print(f"✅ Załadowano: {len(op)} wyników")
    if len(op) > 0:
        result = op[0]
        print(f"📂 Pierwszy wynik: {result.name}")
        print(f"📁 Ścieżka zarr: {result.path}")
    else:
        print("❌ Brak wyników w danych")
        op = None
except Exception as e:
    print(f"❌ Nie można załadować danych: {e}")
    print("💡 Zmień ścieżkę zarr_path na prawidłową")
    op = None

## Przygotuj analizator z ulepszoną konfiguracją

In [ ]:
if op is not None:
    # Konfiguracja z nowymi opcjami
    config = ModeVisualizationConfig(
        figsize=(12, 10),
        f_min=0.5,
        f_max=5.0,
        peak_threshold=0.1,
        # ✨ NOWE: Ustawienia animacji
        colormap_animation="balance",  # Mapa cmocean dla animacji
        use_midpoint_norm=True,  # Symetryczna normalizacja
        animation_time_steps=90,  # Więcej klatek = gładsza animacja
        # Standardowe ustawienia wizualizacji
        colormap_magnitude="inferno",
        colormap_phase="hsv",
        show_magnitude=True,
        show_phase=True,
        show_combined=True,
    )

    # Utwórz analizator
    analyzer = FMRModeAnalyzer(
        zarr_path=result.path,
        dataset_name="m",  # Zmień na odpowiedni dataset
        config=config,
        debug=True,
    )

    print(f"✅ Analizator utworzony dla datasetu: {analyzer.dataset_name}")
    print(
        f"📊 Konfiguracja animacji: mapa '{config.colormap_animation}', normalizacja symetryczna: {config.use_midpoint_norm}"
    )
else:
    print("⚠️ Pomiń tę sekcję - brak danych")

## Oblicz mody (jeśli potrzeba)

In [ ]:
if op is not None:
    # Sprawdź czy mody istnieją
    if not analyzer.modes_available:
        print("🔄 Obliczanie modów FMR...")
        analyzer.compute_modes(window=True, save=True)
        print("✅ Mody obliczone")
    else:
        print("✅ Mody już istnieją")

    # Znajdź piki
    peaks = analyzer.find_peaks()
    print(f"🎯 Znaleziono {len(peaks)} pików:")
    for i, peak in enumerate(peaks[:5]):  # Pokaż pierwsze 5
        print(f"   {i + 1}. {peak.freq:.3f} GHz (amplituda: {peak.amplitude:.3f})")
else:
    print("⚠️ Pomiń tę sekcję - brak danych")

## 🎬 Animacja 1: Temporal (Prawdziwa fizyczna dynamika)

Pokazuje jak mod rzeczywiście oscyluje w czasie: `amplitude * cos(phase + ωt)`

In [ ]:
if op is not None and len(peaks) > 0:
    # Wybierz pierwszą rezonansową częstotliwość
    target_freq = peaks[0].freq
    print(f"🎯 Tworzenie animacji temporalnej dla {target_freq:.3f} GHz")
    print("⏱️  To potrwa kilka sekund...")

    # ✨ NOWA: Temporal animation z fizyczną dynamiką
    analyzer.save_modes_animation(
        frequency=target_freq,  # Stała częstotliwość
        save_path="temporal_balance.gif",  # Plik wyjściowy
        animation_type="temporal",  # Typ: oscylacje w czasie
        fps=15,  # Klatki na sekundę
        z_layer=0,  # Warstwa Z
        component="z",  # Komponent magnetyzacji
        # ✨ NOWE: Zaawansowane opcje
        colormap="balance",  # Mapa cmocean
        use_midpoint_norm=True,  # Symetryczna normalizacja
        figsize=(10, 8),  # Rozmiar figury
    )

    print("✅ Animacja temporalna zapisana jako 'temporal_balance.gif'")
    print("🔬 Ta animacja pokazuje prawdziwą fizyczną dynamikę modu!")
else:
    print("⚠️ Pomiń - brak danych lub pików")

## 🎨 Test różnych map kolorów

In [ ]:
if op is not None and len(peaks) > 0 and CMOCEAN_AVAILABLE:
    target_freq = peaks[0].freq

    # Test różnych map kolorów cmocean
    cmocean_maps = {
        "balance": "Symetryczna - idealna dla oscylacji",
        "diff": "Różnice - dobra dla zmian",
        "curl": "Curl - dobra dla rotacji",
        "delta": "Delta - odchylenia od średniej",
    }

    for cmap_name, description in cmocean_maps.items():
        print(f"🎨 Tworzenie animacji z mapą '{cmap_name}' ({description})")

        analyzer.save_modes_animation(
            frequency=target_freq,
            save_path=f"temporal_{cmap_name}.gif",
            animation_type="temporal",
            fps=10,  # Szybciej dla testów
            component="z",
            colormap=cmap_name,
            use_midpoint_norm=True,
        )

        print(f"✅ Zapisano 'temporal_{cmap_name}.gif'")

    print("\n🎉 Wszystkie mapy kolorów przetestowane!")
    print("💡 Porównaj pliki .gif żeby zobaczyć różnice")
else:
    print("⚠️ Pomiń - brak cmocean lub danych")

## 🎬 Animacja 2: Frequency Sweep

Pokazuje jak zmienia się amplituda modu w funkcji częstotliwości

In [ ]:
if op is not None and len(peaks) > 0:
    print("🎯 Tworzenie animacji frequency sweep")

    # Znajdź zakres częstotliwości wokół pików
    freq_min = max(peaks[0].freq - 0.5, config.f_min)
    freq_max = min(peaks[0].freq + 0.5, config.f_max)

    print(f"📊 Zakres częstotliwości: {freq_min:.3f} - {freq_max:.3f} GHz")

    analyzer.save_modes_animation(
        frequency_range=(freq_min, freq_max),  # Zakres częstotliwości
        save_path="frequency_sweep.gif",
        animation_type="frequency",  # Typ: zmiana częstotliwości
        fps=8,  # Wolniej dla lepszej widoczności
        component="z",
        colormap="inferno",  # Dobra mapa dla amplitudy
    )

    print("✅ Animacja frequency sweep zapisana jako 'frequency_sweep.gif'")
else:
    print("⚠️ Pomiń - brak danych")

## 🎬 Animacja 3: Phase Evolution

Pokazuje ewolucję fazy przy stałej częstotliwości

In [ ]:
if op is not None and len(peaks) > 0:
    target_freq = peaks[0].freq
    print(f"🎯 Tworzenie animacji phase evolution dla {target_freq:.3f} GHz")

    analyzer.save_modes_animation(
        frequency=target_freq,
        save_path="phase_evolution.gif",
        animation_type="phase",  # Typ: ewolucja fazy
        fps=12,
        component="z",
        # Faza zawsze używa HSV colormap - jest to hardcoded w implementacji
    )

    print("✅ Animacja phase evolution zapisana jako 'phase_evolution.gif'")
    print(
        "🌈 Kolory HSV przedstawiają fazę: czerwony=0, żółty=π/2, cyan=π, magenta=3π/2"
    )
else:
    print("⚠️ Pomiń - brak danych")

## 🧪 Test klasy MidpointNormalize

In [ ]:
# Test nowej klasy MidpointNormalize
print("🧪 Test klasy MidpointNormalize")

# Utwórz testowe dane
vals = np.array([[-5.0, 0], [5, 10]])
vmin = vals.min()
vmax = vals.max()

print(f"📊 Dane testowe: min={vmin}, max={vmax}")

# Test z midpoint=0
norm_mid = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)
normalized = norm_mid(vals)

print(f"✅ MidpointNormalize z midpoint=0:")
print(f"   Wejście: {vals.flatten()}")
print(f"   Wyjście: {normalized.data.flatten()}")
print(f"   Punkt środkowy (0) mapuje się na: {norm_mid(0)}")

# Test standardowej normalizacji
norm_std = plt.Normalize(vmin=vmin, vmax=vmax)
normalized_std = norm_std(vals)

print(f"\n📊 Standardowa normalizacja:")
print(f"   Wejście: {vals.flatten()}")
print(f"   Wyjście: {normalized_std.flatten()}")
print(f"   Punkt 0 mapuje się na: {norm_std(0)}")

print(
    "\n💡 MidpointNormalize zapewnia, że midpoint (0) zawsze mapuje się na środek mapy kolorów (0.5)"
)
print("   To jest idealne dla danych oscylujących wokół zera!")

## 📋 Podsumowanie

### ✅ Co zostało ulepszone:

1. **Fizycznie poprawna animacja temporalna**:
   - `real_part = amplitude * cos(phase + ωt)`
   - Pokazuje prawdziwą dynamikę modu w czasie

2. **Klasa MidpointNormalize**:
   - Symetryczna normalizacja wokół wybranego punktu (domyślnie 0)
   - Idealna dla danych oscylujących +/-
   - Automatycznie włączana dla animacji temporalnych

3. **Mapy kolorów cmocean**:
   - `balance` - symetryczna, idealna dla oscylacji
   - `diff` - dobra dla różnic/zmian
   - `curl` - dobra dla rotacji/fazy
   - `delta` - odchylenia od średniej
   - `tarn` - dane zespolone
   - Fallbacki matplotlib gdy cmocean niedostępne

4. **Inteligentne domyślne**:
   - Auto-wybór `MidpointNormalize` dla animacji temporalnych
   - Lepsze domyślne mapy kolorów dla każdego typu animacji
   - Informacyjne komunikaty o wyborach

### 🎯 Użycie:

```python
# Prawdziwa fizyczna animacja
analyzer.save_modes_animation(
    frequency=1.5,                    # GHz
    animation_type="temporal",        # Oscylacje w czasie
    colormap="balance",               # Symetryczna mapa
    use_midpoint_norm=True           # Auto-włączone dla temporal
)

# Przez interfejs FFT
result.fft.save_modes_animation(
    frequency=1.5,
    animation_type="temporal"
)
```